In [82]:
ll = ["l1", "l2", "l3"]

print("from left0", ll[0:])
print("from left1", ll[1:])
print("from left2", ll[2:])

print("from right0", ll[:0])
print("from right1", ll[:1])
print("from right2", ll[:2])

from left0 ['l1', 'l2', 'l3']
from left1 ['l2', 'l3']
from left2 ['l3']
from right0 []
from right1 ['l1']
from right2 ['l1', 'l2']


In [6]:
from pathlib import Path
import json

fl=Path("converted-tcs.json")

print(f"is this, {fl} , file?", fl.is_file())

d=dict()
with fl.open("r", encoding="utf-8") as f:
    print("file content")
    d=json.loads(f.read())

print("isDictionary?", isinstance(d,dict), "dictionary: ", d)
if isinstance(d,dict):
    print("length of keys: ", len(d.keys()), "keys: ", d.keys())

is this, converted-tcs.json , file? True
file content
isDictionary? True dictionary:  {'view-expense-list-tc1': {'metadata': {'id': 'view-expense-list-tc1', 'relative_file_path': 'expense\\view-expense-list-tc1.md', 'file_name': 'view-expense-list-tc1.md'}, 'details': {'Title': 'View expense list', 'Description': 'logged in user can view list of expenses and gets actions for each expense if applicable', 'Preconditions': 'user is logged in and is on homepage.', 'Steps to Execute': ['user clicks on expense navigation link', 'after few seconds of waiting, list of expenses of recent 3 months displayed.', '3 add buttons `add purchase`, `add income` and `add refund` are displayed before expense list layout.', 'verify fields and actions as per expected.', 'user clicks on `load more` button.', 'after waiting, more expenses (3 more months) are loaded. recent 6 months of expenses.', 'user clicks on `load more` button.', 'after waiting, `load more` button gets disabled, since there are no more ex

In [27]:
tcd = d if isinstance(d,dict) else dict()

#  list features
# create a map , feature as key and test case id as value in list
feature_dict= dict()
impact_dict= dict()
area_dict= dict()
test_type_dict= dict()

for tck,tcv in tcd.items():
    if isinstance(tcv, dict):
        for fv in tcv["details"]["Tags"]["feature"]:
            if fv not in feature_dict:
                feature_dict[fv] = list()
            feature_dict[fv].append(tck)
        for iv in tcv["details"]["Tags"]["impact"]:
            if iv not in impact_dict:
                impact_dict[iv] = list()
            impact_dict[iv].append(tck)
        for iak,iav in tcv["details"]["Impact Area"].items():
            if iak not in area_dict:
                area_dict[iak] = list()
            area_dict[iak].append(tck)
            for iavv in iav:
                ak = f"{iak}+{iavv}"
                if ak not in area_dict:
                    area_dict[ak] = list()
                area_dict[ak].append(tck)
        for ttv in tcv["details"]["Type of Test"]:
            if ttv not in test_type_dict:
                test_type_dict[ttv] = list()
            test_type_dict[ttv].append(tck)
        

print("how many features are there? ", len(feature_dict.keys()), "list of feature=", feature_dict.keys())

# print("feature dict=", json.dumps(feature_dict, indent=4))

print("how many impact levels are there? ", len(impact_dict.keys()), "list of imact levels=", impact_dict.keys())

# print("impact dict=", json.dumps(impact_dict, indent=4))

print("how many impacted areas are there? ", len(area_dict.keys()), "list of imacted area=", area_dict.keys())

print("how many test types are there? ", len(test_type_dict.keys()), "list of test types=", test_type_dict.keys())




how many features are there?  12 list of feature= dict_keys(['expense', 'view-list', ' receipt', 'income', 'add', 'edit', 'delete', 'purchase', 'refund', 'login', 'logout', 'signup'])
how many impact levels are there?  3 list of imact levels= dict_keys(['high', 'medium', 'low'])
how many impacted areas are there?  42 list of imacted area= dict_keys(['frontend', 'frontend+expense page', 'backend api', 'backend api+expense list api', 'backend api+purchase type api', 'backend api+refund reason api', 'backend api+payment account api', 'backend api+income type api', 'frontend+add income form', 'frontend+update income form', 'backend api+share person api', 'backend api+income tags api', 'backend api+income api post', 'frontend+delete income', 'backend api+income api delete', 'backend api+income api get', 'frontend+add purchase form', 'frontend+update purchase form', 'backend api+purchase tags api', 'backend api+purchase api post', 'frontend+delete purchase', 'backend api+purchase api delete'

In [83]:
from typing import Dict, List

def traverse_dict(key_depths:List[str], wrapper: Dict[str,Dict]):
    if len(key_depths) == 0:
        return wrapper
    
    for k,v in wrapper.items():
        if k.lower() == key_depths[0].lower():
            return traverse_dict(key_depths[1:], v)

def print_summary(search_key:List[str], sd:Dict, tc_key:str, tc_details:Dict[str,Dict]):
    # print("keys=", search_key, len(search_key))
    v = traverse_dict(search_key,tc_details)
    append_list_summary("", v, sd, tc_key)
    append_dict_summary(v, sd, tc_key)

    # for k,v in tc_details.items():
        # if len(search_key)>0 and k.lower() == search_key[0].lower():
            # print("k=", k, "v=", v)
            # for k1,v1 in v.items():
                # print("k1", k1, "search", search_key[1])
                # if len(search_key)>1 and k1.lower() == search_key[1].lower():
                    # print("k1=", k1, "v1=", v1)
                    # append_list_summary("", v1, sd, tc_key)
                    # append_dict_summary(v1, sd, tc_key)

def append_list_summary(key_prefix:str, list_values: List, sd:Dict[str,List], tc_key:str):
     if isinstance(list_values, list):
        for lv in list_values:
            if isinstance(lv, str):
                skey = f"{key_prefix}+{lv}" if len(key_prefix) > 0 else lv
                if skey not in sd:
                    sd[skey] = list()
                sd[skey].append(tc_key)
            if isinstance(lv, list):
                append_list_summary(key_prefix, lv, sd, tc_key)

def append_dict_summary(dict_value:Dict, sd:Dict[str,List],tc_key:str):
    if isinstance(dict_value, dict):
        for k2,v2 in dict_value.items():
            if k2 not in sd:
                sd[k2] = list()
            sd[k2].append(tc_key)
            append_list_summary(k2, v2, sd, tc_key)

def print_test_cases_summary(search_keys:str, converted_tc_details: Dict[str, Dict]):
    summary_dict = dict()

    for tck,tcv in converted_tc_details.items():
        skeys = search_keys.split(".")
        print_summary(skeys, summary_dict, tck, tcv)
    
    search_term = search_keys.split(".")[-1:][0]
    print(f"how many '{search_term}' are there?", len(summary_dict))

    print(f"list of '{search_term}'", json.dumps(list(summary_dict.keys())))

    print(f"'{search_term}' json", json.dumps(summary_dict, indent=4))


print_test_cases_summary("details.Type of Test", tcd)

print_test_cases_summary("details.Impact Area", tcd)

print_test_cases_summary("details.Tags.impact", tcd)

print_test_cases_summary("details.Tags.feature", tcd)

# lsr=search_in.split(".")
# print("reverse=", lsr, lsr[1:][0])


# details.Type of Test
# 
# {tcid: {}}
# tcv dict = {details: {}}
# tcv dict = {type of test: []}
# 


how many 'Type of Test' are there? 4
list of 'Type of Test' ["Integration", "Regression", "Demo site testable", "Feature"]
'Type of Test' json {
    "Integration": [
        "view-expense-list-tc1",
        "view-receipts-expenses-tc2",
        "add-edit-income-tc4",
        "add-income-tc1",
        "delete-income-tc5",
        "edit-income-tc2",
        "edit-income-tc3",
        "add-edit-purchase-tc4",
        "add-purchase-tc1",
        "delete-purchase-tc5",
        "edit-purchase-tc2",
        "edit-purchase-tc3",
        "add-edit-refund-tc4",
        "add-refund-tc1",
        "add-refund-tc2",
        "delete-refund-tc6",
        "edit-refund-tc3",
        "edit-refund-tc4",
        "login-logout-tc1",
        "login-tc1",
        "login-tc2",
        "login-tc3",
        "login-tc4",
        "logout-tc1",
        "logout-tc2",
        "signup-tc1",
        "signup-tc2",
        "signup-tc3"
    ],
    "Regression": [
        "view-expense-list-tc1",
        "view-receipts-exp